# Real estate ROI calculator 

In [1]:
import urllib.request
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Selector
import selenium
from selenium import webdriver
import chromedriver_binary
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
from crochet import setup
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 200)

import json
import re
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

In [2]:
import datetime

location = "bayern/fuerth-kreis"
propertyType = 'wohnung'

#url_to_crawl = 'https://www.immobilienscout24.de/Suche/de/' + location + '/'+propertyType+'-kaufen?enteredFrom=result_list'

url_to_crawl = 'https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?enteredFrom=one_step_search'

actualDate = datetime.date.today()
actualDate = actualDate.strftime('%Y-%m-%d')
city = location.split('/')[1]

# Your variables to change
rent_price_index = 9
equity_percentage = 0.20
interest_rate = 0.01
mortgage_payment_rate = 0.02

# Standard percentages
notary_percentage = 0.015
transfer_tax_percentage = 0.05
land_registry_tax_percentage = 0.005
additional_costs_rate_per_m2 = 0.9 

In [3]:

columns = ['id_expose', 'url', 'title', 'address', 'contact_person', 'telephone', 'mobile', 'fax', 'price_purchase', 'estimated_monthly_rate', 'rooms', 'living_area_m2', 'furnishing', 'flat_type', 'floor', 'floor_total', 'bedrooms', 'commission_buyer_rate', 'maintenance_cost', 'construction_year', 'object_state', 'heating_type', 'energy_efficiency_class', 'location']
df_base = pd.DataFrame(columns = columns)

options = Options()
options.headless = True

crawler_name = 'scrapper_immobilienscout'

base_uri = 'immobilienscout24.de'
base_url = 'https://www.immobilienscout24.de'

In [4]:
driver = webdriver.Chrome()
# OLD configuration
#driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")

class Scrap_immobilienscout(scrapy.Spider):
    
    name = crawler_name
    
    def start_requests(self):
        first_page = urllib.request.urlopen(url_to_crawl)
        max_pages = first_page.read()
        max_pages = max_pages.decode("utf8")
        first_page.close()

        max_pages_selector = Selector(text = max_pages)
        max_pages = max_pages_selector.xpath('//select[@aria-label="Seitenauswahl"]/option[last()]/@value').extract_first()
        max_pages = int(max_pages) if max_pages is not None else 1
        
        base_url = url_to_crawl.replace('enteredFrom=one_step_search', '')
        
        for page_number in range(max_pages):
            url = base_url + 'pagenumber=' + str(page_number + 1)
            yield scrapy.Request(url, callback = self.parse_properties_list)
    
    def parse_properties_list(self, response):
        
        property_urls = response.xpath('//a[contains(@class, "result-list-entry__brand-title-container")]/@href').extract()
        
        for url in property_urls:
            if base_uri not in url:
                url = base_url + url + '#/'
                driver.get(url)
                src = driver.page_source
                self.scrap_property(src, url)
    
    def scrap_property(self, src, url):
        sel = Selector(text = src)

        id_expose = '' # TODO get expose substring of url
        
        title = sel.css('h1#expose-title ::text').extract_first()
        
        address = sel.xpath('//div[@class="address-block"]//span[@class="block font-nowrap print-hide"]//text()').extract_first()
        address = address.strip() if address is not None else ''

        region = sel.css('span.zip-region-and-country::text').extract_first()
        address += ' ' + region if region is not None else ''
        
        contact_person = sel.xpath('//div[@data-qa="contactName"]//text()').extract_first()
        contact_person = contact_person if contact_person is not None else ''
        
        phone = sel.xpath('//script//text()').extract_first()
        phone = phone if phone is not None else ''

        telefon = re.findall('"phoneNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        telefon = telefon[0].replace(' ', '') if len(telefon) > 0 else None
        
        mobil = re.findall('"cellPhoneNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        mobil = mobil[0] if len(mobil) > 0 else ''
        mobil = mobil.replace(' ', '')

        fax = re.findall('"faxNumber":\{"contactNumber":"(\+?[0-9 ]*)"\}', phone)
        fax = fax[0] if len(fax) > 0 else ''
        fax = fax.replace(' ', '')
        
        price = sel.xpath('//dd[contains(@class, "is24qa-kaufpreis")]//text()').extract_first()
        price = re.sub('[^0-9]', '', price) if price is not None else 0
        price = int(price)
        
        estimated_monthly_rate = sel.xpath('//span[contains(@class, "monthly-rate-result") and contains(@class, "monthly-rate-value")]//text()').extract_first()
        estimated_monthly_rate = re.sub('[^0-9]', '', estimated_monthly_rate) if estimated_monthly_rate is not None else 0
        estimated_monthly_rate = int(estimated_monthly_rate)
        
        rooms_count = sel.xpath('//dd[contains(@class,"is24qa-zimmer")]/text()').extract_first()
        rooms_count = float(rooms_count.replace(" ","").replace(",",".") ) if rooms_count is not None else 0
        
        living_area = sel.xpath('//dd[contains(@class, "is24qa-wohnflaeche-ca")]/text()').extract_first()
        living_area = re.sub('[^0-9\.]', '', living_area.replace(",",".")) if living_area is not None else 0
        living_area = float(living_area)
        
        furnishing = sel.xpath('//div[contains(@class, "criteriagroup") and contains(@class, "boolean-listing")]//span[contains(@class, "palm-hide")]/text()').extract()
        furnishing = list(set(furnishing))
        furnishing = [t for t in furnishing if t != ' ']
        furnishing = furnishing if len(furnishing) > 0 else ''
        
        flat_type = sel.xpath('//dd[contains(@class, "is24qa-typ")]/text()').extract_first()
        
        floor = sel.xpath('//dd[contains(@class, "is24qa-etage")]//text()').extract_first()
        floor = floor if floor is not None else ''
        floor = re.findall('[0-9]+', floor)
        total_floors = int(floor[1]) if len(floor) > 1 else 0
        floor = int(floor[0]) if len(floor) > 0 else 0
        
        bedroom = sel.xpath('//dd[contains(@class, "is24qa-schlafzimmer")]//text()').extract_first()
        bedroom = re.sub('[^0-9]', '', bedroom) if bedroom is not None else 0
        bedroom = int(bedroom)
        
        allowance = sel.xpath('//dd[contains(@class, "is24qa-hausgeld")]//text()').extract_first()
        allowance = re.sub('[^0-9]', '', allowance) if allowance is not None else 0
        allowance = int(allowance)
        
        commission = sel.xpath('//dd[contains(@class, "is24qa-provision")]/text()').extract_first()
        
        if commission is not None:
            try:
                commission = re.search('[0-9]+,?[0-9]*', commission).group() 
            except:
                commission = ''
        else:
            commission = ''

        commission = commission.replace(',', '.')
        commission = float(commission) if len(commission) > 0 else 0
        
        construction_year = sel.xpath('//dd[contains(@class, "is24qa-baujahr")]//text()').extract_first()
        
        object_state = sel.xpath('//dd[contains(@class, "is24qa-objektzustand")]//text()').extract_first()
        
        heatingy_type = sel.xpath('//dd[contains(@class, "is24qa-heizungsart")]//text()').extract_first()
        
        energy_efficiency_class = sel.xpath('//dd[contains(@class, "is24qa-energieeffizienzklasse")]//text()').extract_first()
        energy_efficiency_class = energy_efficiency_class if energy_efficiency_class is not None else ''
        
        location = sel.xpath('//pre[contains(@class, "is24qa-lage")]/text()').extract_first()

        lst_row = [id_expose, url, title, address, contact_person, telefon, mobil, fax, price, estimated_monthly_rate, rooms_count, living_area, furnishing, flat_type, floor, total_floors, bedroom, commission, allowance, construction_year, object_state, heatingy_type, energy_efficiency_class, location]
        df_base.loc[len(df_base)] = lst_row
        df_base.to_csv(file_path, index=False)
        
setup()
def spider_process(spider):
    process = CrawlerProcess()
    process.crawl(spider)
    
spider_process(Scrap_immobilienscout)
# TODO unblock process. See: https://docs.scrapy.org/en/latest/topics/practices.html 

2020-07-22 17:18:09 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: scrapybot)
2020-07-22 17:18:09 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Apr 18 2020, 01:56:04) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-5.3.0-62-generic-x86_64-with-Ubuntu-18.04-bionic
2020-07-22 17:18:09 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-07-22 17:18:09 [scrapy.crawler] INFO: Overridden settings:
{}
2020-07-22 17:18:09 [scrapy.extensions.telnet] INFO: Telnet Password: 067fd2d936c589d1
2020-07-22 17:18:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-07-22 17:18:09 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy

2020-07-22 17:18:15 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 200 14
2020-07-22 17:18:15 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:15 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/source {}
2020-07-22 17:18:15 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "GET /session/bd1dfdb70157c1c80ce2d28022597e9f/source HTTP/1.1" 200 831809
2020-07-22 17:18:15 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:15 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=5> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *arg

2020-07-22 17:18:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=13> (referer: None)
2020-07-22 17:18:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=12> (referer: None)
2020-07-22 17:18:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=14> (referer: None)
2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "https://www.immobilienscout24.de/expose/120371388#/"}
2020-07-22 17:18:17 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 404 222
2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:17 [scrapy.core.scraper] ERROR: 

2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "https://www.immobilienscout24.de/expose/114665751#/"}
2020-07-22 17:18:17 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 404 222
2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:17 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=14> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-4-af57f78f4e24>", line 32, in parse_properties_list
    driver.get(url)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/r

2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "https://www.immobilienscout24.de/expose/113133513#/"}
2020-07-22 17:18:17 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 404 222
2020-07-22 17:18:17 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:17 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=19> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-4-af57f78f4e24>", line 32, in parse_properties_list
    driver.get(url)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/r

...

2020-07-22 17:18:19 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=61> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-4-af57f78f4e24>", line 32, in parse_properties_list
    driver.get(url)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.comm

2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:20 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=67> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-4-af57f78f4e24>", line 32, in parse_properties_list
    driver.get(url)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/remote/errorhandler.py", line 2

2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "https://www.immobilienscout24.de/expose/116794750#/"}
2020-07-22 17:18:20 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 404 222
2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:20 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=72> (referer: None)
Traceback (most recent call last):
  File "/home/promit/.local/lib/python3.6/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "<ipython-input-4-af57f78f4e24>", line 32, in parse_properties_list
    driver.get(url)
  File "/home/promit/.local/lib/python3.6/site-packages/selenium/webdriver/r

2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=77> (referer: None)
2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=81> (referer: None)
2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=80> (referer: None)
2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=78> (referer: None)
2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=79> (referer: None)
2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "

2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=86> (referer: None)
2020-07-22 17:18:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=87> (referer: None)
2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:57829/session/bd1dfdb70157c1c80ce2d28022597e9f/url {"url": "https://www.immobilienscout24.de/expose/112892928#/"}
2020-07-22 17:18:20 [urllib3.connectionpool] DEBUG: http://127.0.0.1:57829 "POST /session/bd1dfdb70157c1c80ce2d28022597e9f/url HTTP/1.1" 404 222
2020-07-22 17:18:20 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2020-07-22 17:18:20 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.immobilienscout24.de/Suche/de/berlin/berlin/wohnung-kaufen?pagenumber=82> (referer: None)
Traceback (most recent call last):
  F

In [6]:
df_base.head()

,id_expose,url,title,address,contact_person,telephone,mobile,fax,price_purchase,estimated_monthly_rate,rooms,living_area_m2,furnishing,flat_type,floor,floor_total,bedrooms,commission_buyer_rate,maintenance_cost,construction_year,object_state,heating_type,energy_efficiency_class,location
0,,https://www.immobilienscout24.de/expose/117438016#/,Einzigartiges 5-Zimmer Penthouse mit 2 Terrassen und 2 Kaminen nahe Messelpark,"14195 Berlin, Schmargendorf (Wilmersdorf)",Herr Peter Rabitz,01727999170,,,1998000,0,5.0,185.00,"[Einbauküche, Gäste-WC, Keller, Balkon/ Terrasse]",Penthouse,2,3,3,7.14,0,1979,Gepflegt,Fußbodenheizung,F,Das Wohnhaus befindet sich in der traditionellen Villengegend Berlin Schmargendorf/Dahlem am Messelpark und unweit des beliebten Rosenecks mit seinen Einkaufsmöglichkeiten. Prächtige klassische Vi...
1,,https://www.immobilienscout24.de/expose/116610688#/,3-Zimmer-Altbauwohnung in begehrter Lage zum Selbstbezug,"Torstraße 225B, 10117 Berlin, Mitte (Mitte)",Frau Birgit Ahrends,03071817552,,030887181135,740000,0,3.0,85.99,"[Provisionsfrei, Personenaufzug, Keller, Balkon/ Terrasse]",Etagenwohnung,4,5,2,0.00,26286,1900,Erstbezug nach Sanierung,Zentralheizung,G,Wie der Name ‚Berlin-Mitte‘ verspricht: Hier lebt man mitten im Herzen der Stadt! Mitte setzt sich aus einer Vielzahl an lebendigen Kiezen mit eigenem Charakter zusammen. Die Torstraße liegt in ei...
2,,https://www.immobilienscout24.de/expose/116794278#/,"Smartes Investment: Mietrenditen von bis zu 4,5%","10315 Berlin, Friedrichsfelde (Lichtenberg)",Ihr Best Place Vertriebsteam,03044351960,,,208000,0,1.0,29.96,"[Einbauküche, Balkon/ Terrasse, Personenaufzug, Keller, Online-Besichtigung möglich, Provisionsfrei]",Etagenwohnung,3,6,0,0.00,0,2021,None,None,,"Raus aus der Tür und rein ins bunte Leben. Nur wenige Meter sind es bis zum angesagten Weitlingkiez mit einer bunten Vielfalt an Cafés, Bistros und kleinen Läden. Das Leben in Berlin ist hier pure..."
3,,https://www.immobilienscout24.de/expose/116021017#/,"Mitten in Mitte, aber Ruhe wie draußen!","10178 Berlin, Mitte (Mitte)",Herr Michael Werner,0306566020,,03065660217,300000,0,3.0,59.00,[Einbauküche],Etagenwohnung,1,0,0,3.48,210,1962,Gepflegt,Fernwärme,B,Das Objekt befindet sich nur wenige Gehminuten vom Alexanderplatz entfernt. Die nächste Tram Haltestelle Mollstrasse ist in ca. 5 Minuten fussläufig zu erreichen. Hier verkehren zahlreiche Bahn/ -...
4,,https://www.immobilienscout24.de/expose/111313235#/,"Zuhause in Westend: großzügige 3-Zimmer-Wohnung mit eigenem, geschütztem Garten und Loggia","Ahornallee 21, 14050 Berlin, Charlottenburg (Charlottenburg)",Niederlassung 1 Berlin,None,,,79420850,0,3.0,95.71,"[Provisionsfrei, Personenaufzug, Gäste-WC, Balkon/ Terrasse]",Etagenwohnung,0,0,2,0.00,0,2019,Erstbezug,None,B,"Treten Sie in die Fußstapfen von Marlene Dietrich und Erich Maria Remarque und begeben Sie sich auf eine Reise durch ihre vielfältige Nachbarschaft – hier gibt es viel zu entdecken! Ob Kunst, Kult..."


In [7]:
df_base[['price_purchase', 'estimated_monthly_rate', 'floor', 'floor_total', 'bedrooms', 'maintenance_cost']] = df_base[['price_purchase', 'estimated_monthly_rate', 'floor', 'floor_total', 'bedrooms', 'maintenance_cost']].astype(float)

In [8]:
df_finance = df_base[['url','price_purchase', 'living_area_m2', 'commission_buyer_rate']]
df_finance['price_per_m2'] = df_finance['price_purchase'] / df_finance['living_area_m2']
df_finance['notary_costs'] = notary_percentage * df_finance['price_purchase']

df_finance['land_registry_tax'] = land_registry_tax_percentage * df_finance['price_purchase']
df_finance['real_estate_transfer_tax'] = transfer_tax_percentage * df_finance['price_purchase']
df_finance['commission'] = df_finance['price_purchase'] * df_finance['commission_buyer_rate'] * 0.01 # TODO fix while parsing from scraper already
df_finance['total_buying_price'] = df_finance['price_purchase'] + df_finance['notary_costs'] + df_finance['land_registry_tax'] + df_finance['real_estate_transfer_tax'] + df_finance['commission']
df_finance['equity'] = equity_percentage * df_finance['total_buying_price']
df_finance['debt'] = df_finance['total_buying_price'] - df_finance['equity']
df_finance['interest_rate_monthly'] = interest_rate * df_finance['debt'] / 12
df_finance['mortgage_payment_yearly'] = mortgage_payment_rate * df_finance['debt']
df_finance['mortgage_payment_monthly'] = df_finance['mortgage_payment_yearly'] / 12
df_finance['monthly_rent'] = rent_price_index * df_finance['living_area_m2']
df_finance['additional_costs'] = additional_costs_rate_per_m2 * df_finance['living_area_m2']
df_finance['cashflow'] = df_finance['monthly_rent'] - df_finance['interest_rate_monthly'] - df_finance['mortgage_payment_monthly'] - df_finance['additional_costs']
df_finance['ROI'] = (df_finance['cashflow'] * 12) / df_finance['equity'] 





2020-07-22 17:26:16 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.
2020-07-22 17:26:16 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

2020-07-22 17:26:16 [py.warnings] WARNING: /home/promit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until

2020-07-22 17:26:16 [py.warnin